# Como Tratar Dados Ausentes com Pandas

Uma das maiores dificuldades da etapa exploratória dos dados em um projeto de Data Science é limpar os dados, o que envolve lidar com dados faltantes.

Durante a fase em que estamos aprendendo, lendo livros e seguindo tutoriais, não temos tanto esse tipo de trabalho, pois os dados já vêm mais "redondinhos".

Entretanto, essa não é a realidade do mundo real, onde a gente vai se deparar com uma infinidade de problemas e *missing data*.

Excluir, completar ou ignorar?! Qual a melhor abordagem?

Hoje nós vamos falar sobre as principais técnicas para lidar com dados ausentes e entender o *trade-off* entre elas.

## Identificando dados ausentes

Algoritmos de *Machine Learning* não são capazes de lidar com valores ausentes (*missing data*). Se você já tentou rodar, viu que a mensagem de erro é bem clara em relação a isso.

Um valor ausente é facilmente identificado nos campos da sua estrutura de dados como ```NaN```. Para seu modelo rodar sem problemas, você tem que limpar os dados (*data cleaning*) em uma etapa anterior. É aí que surge a dúvida: o que eu faço com eles?

Não existe uma resposta 100% correta, pois cada abordagem tem suas vantagens e desvantagens, mas vamos dar uma olhada naquelas que são mais usadas.

Para isso, vamos importar um conjunto de dados do [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) para usar de exemplo.

### Importando os dados

Vou usar o conjunto de dados [House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) neste exemplo, pois ele apresenta várias colunas com valores inexistentes.

Nesse *dataset*, temos 80 variáveis (colunas) e 1460 entradas (linhas) no arquivo **train.csv**. Eu vou baixar apenas esse arquivo, e vou usar a (Kaggle API)(https://www.kaggle.com/docs/api), uma vez que ela me permite baixar o arquivo diretamente do Terminal.

Se você não conhece a API, (olhe a documentação no Github)[https://github.com/Kaggle/kaggle-api]. Aprender a usar essa API vai te ajudar muito, não apenas pela facilidade, mas porque deixa seu *notebook* replicável para quem clonar ele.

In [1]:
!mkdir -p data
!kaggle competitions download -c house-prices-advanced-regression-techniques -f train.csv -p data

train.csv: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
# importar o Pandas
import pandas as pd

# importar train.csv em DataFrame
df = pd.read_csv('data/train.csv')

# visualizar as 5 primeiras entradas
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Identificando dados ausentes

A primeira coisa que você tem que saber quando lida com um conjunto de dados novos, é a quantidade e proporção dos *missing values*.

Para identificar valores ausentes, por colunas, você pode usar:

* ```df.describe()``` para retornar um resumo estatístico das variáveis numéricas
* ```df.info()``` para dar um resumo de valores não-nulos encontrados
* ```df.isnull().sum()``` para retornar a soma dos valores nulos encontrados

Usando essa última opção, vamos ver as 10 colunas com mais valores ausentes para esse *dataset*.

In [3]:
df.isnull().sum().sort_values(ascending=False)[:10]

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageCond        81
GarageType        81
GarageYrBlt       81
GarageFinish      81
dtype: int64

## Lidando com dados ausentes

Como eu falei lá em cima, não existe uma resposta 100% correta ou 100% errada sobre como você deve tratar os valores ausentes do seu conjunto de dados.

Toda escolha tem uma renúncia. Você tem que estar ciente disso e testar o que vai se adequar melhor àquela situação.

Vou passar aqui alguns dos métodos mais frequentementes usados por cientistas de dados, e como você poderia aplicar esses métodos nos dados que acabamos de importar.

### Excluir valores ausentes

Esta é uma decisão mais radical, e deve ser feita apenas em casos onde não haverá impacto significativo no modelo. Ao eliminar uma linha inteira você joga fora um monte de informação que poderia ser extremamente importante.

Eu uso essa opção apenas quando meu *dataset* é consideravelmente grande e a quantidade de valores ausentes é relativamente insignificante.

Para fazer isso, você vai usar o método ```df.dropna()```. Esse método é direto, e remove os valores ```NaN``` encontrados no *DataFrame*.

Por padrão, se você não informar o eixo, serão eliminadas todas as linhas relativas à celula contendo o valor ausente (```df.dropna(axis=0)```). 

Caso você deseje eliminar uma coluna inteira onde existam ```NaN```, você deve informar explicitamente com ```df.dropna(axis=1)``` 

No exemplo abaixo, todos os valores ```NaN``` da coluna **GarageFinish** serão eliminados. Compare o *shape* antes e depois.

In [4]:
# ver o shape antes
shape_antes = df.shape
print("Antes:\t", shape_antes)

df.dropna(subset=['GarageCond'], inplace=True)

# ver o shape depois
shape_depois = df.shape
print("Depois:\t", shape_depois)

Antes:	 (1460, 81)
Depois:	 (1379, 81)


### Preencher valores ausentes

Esta abordagem é a mais utilizada, pois você não joga fora informação útil. Aqui, a dúvida que você vai ter é em relação a qual valor usar para preencher os dados faltantes.

Existem técnicas avançadas que são combinadas com o preenchimento de valores, como por exemplo analisar correlações ou mesmo construir um modelo preditivo para *missing values*.

Entretanto, uma abordagem direta e simples consiste em substituir os ```NaN``` pela mediana da coluna. Isso é feito mediante o método ```df.fillna()```, informando o valor desejado como argumento.

Vamos extrair a mediana da coluna **LotFrontage** e preencher os valores faltantes com ela.

In [5]:
# extrair a mediana de LotFrontage
lotFrontage_mediana = df.LotFrontage.median()

# preencher a coluna LotFrontage com a mediana
df.fillna(lotFrontage_mediana, inplace=True)

# checar se há valor ausente
df.LotFrontage.isnull().sum()

0

### Usar valor mais frequente

Para preencher *missing values* no último exemplo, usamos a mediana da coluna. Entretanto, caso a variável fosse categórica (e não numérica), poderíamos verificar qual o valor mais frequente e usar ele no preenchimento.

Para identificar o valor mais frequente, basta usar o método ```value_counts()```, extrair o maior valor e informar ele como argumento de ```fillna()```.

In [6]:
# identificar o valor mais frequente
print(df.GarageCond.value_counts())
garageCond_freq = df.GarageCond.value_counts()[0]

# preencher missing values com o valor acima
df.GarageCond.fillna(garageCond_freq, inplace=True)

# verificar se há valor ausente
df.GarageCond.isnull().sum()

TA    1326
Fa      35
Gd       9
Po       7
Ex       2
Name: GarageCond, dtype: int64


0

## Como você pode aplicar esses métodos e melhorar suas análises de *Data Science*?

[Eu vivo falando no meu Instagram](http://instagram.com/carlos_melo.py) sobre a importância da fase de análise de dados em um projeto de *Data Science*.

Aquilo que diferencia um cientista de dados é a sua capacidade de entender, explorar e tratar os dados da melhor maneira possível.

Em *datasets* reais, não tenha dúvida que você vai se deparar com muita inconsistência, lançamentos errados, bases diferentes e muitos valores ausentes.

Saber o que fazer com esses valores vai aumentar muito o desempenho do seu algoritmo de *Machine Learning* e nos seus resultados.

Aproveite para começar a aplicar os métodos que você viu aqui nos seus projetos. Deixe nos comentários abaixo as dicas ou técnicas que você usa nos seus tratamentos.